In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#import plotly.express as px
import tensorflow as tf
import pandas as pd

In [2]:
train_df = pd.read_csv('/home/joaquin/Desktop/Datos/tp2/train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
test_df = pd.read_csv('/home/joaquin/Desktop/Datos/tp2/test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [4]:
#Procesamiento de datos
X_train, X_test, y_train, y_test = train_test_split(train_df['text'], train_df['target'], test_size=0.33,
                                                  random_state=42)
FTest = test_df['text']
test_ids = test_df['id'] 

In [5]:
#Parametros
epochs = 5
maxlen = 40
n_words = 500
embedding_dim = 16

In [6]:
#Tokenización
tokenizer = Tokenizer(num_words = n_words, oov_token = '<OOV>')

tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
sequences_FTest = tokenizer.texts_to_sequences(FTest)


padded_train = pad_sequences(sequences_train, maxlen = maxlen, truncating = 'post')
padded_test = pad_sequences(sequences_test, maxlen = maxlen)
padded_FTest = pad_sequences(sequences_FTest, maxlen = maxlen)

In [7]:
#modelo
model = tf.keras.Sequential([tf.keras.layers.Embedding(n_words,embedding_dim, input_length = maxlen),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model

In [8]:
#entreno
model.fit(padded_train, y_train, epochs=epochs, validation_data = (padded_test, y_test))

Epoch 1/5
160/160 [==============================] - 4s 22ms/step - loss: 0.6588 - accuracy: 0.6014 - val_loss: 0.6206 - val_accuracy: 0.6777
Epoch 2/5
160/160 [==============================] - 4s 24ms/step - loss: 0.5594 - accuracy: 0.7276 - val_loss: 0.5439 - val_accuracy: 0.7394
Epoch 3/5
160/160 [==============================] - 3s 21ms/step - loss: 0.4737 - accuracy: 0.7790 - val_loss: 0.5037 - val_accuracy: 0.7708
Epoch 4/5
160/160 [==============================] - 3s 17ms/step - loss: 0.4269 - accuracy: 0.8090 - val_loss: 0.4955 - val_accuracy: 0.7692
Epoch 5/5
160/160 [==============================] - 3s 17ms/step - loss: 0.3958 - accuracy: 0.8202 - val_loss: 0.5053 - val_accuracy: 0.7684


In [9]:
preds = model.predict_classes(padded_FTest)
#preds = pd.Series(preds)
preds = pd.Series(list((x[0] for x in preds)))
preds

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


0       0
1       1
2       0
3       0
4       1
       ..
3258    0
3259    1
3260    1
3261    0
3262    0
Length: 3263, dtype: int64

In [10]:
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('/home/joaquin/Desktop/Datos/tp2/Keras.csv')
df_preds.head()

,target
id,
0,0
2,1
3,0
9,0
11,1


Tuning: 
1-  epochs = 10
    maxlen = 50
    n_words = 500
    embedding_dim = 16
Acuracy: 0.7441

2-  epochs = 6
    maxlen = 40
    n_words = 500
    embedding_dim = 16
Accuracy : 0.7656b

3-  epochs = 5
    maxlen = 40
    n_words = 300
    embedding_dim = 16
Accuracy: 0.7437
s